# Photometric Calibration

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import(PowerNorm,LogNorm)
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
import os
import scipy.ndimage as snd
from scipy import signal
import glob
import matplotlib.pyplot as plt

In [ ]:
filt= 'B'

In [ ]:
filenames = glob.glob('./Final_Data/Calibration/F66_%s*' % filt)
filenames

In [ ]:
files = [fits.getdata(image) for image in filenames]

In [ ]:
plt.figure(figsize=((15,15)))
plt.subplot(131)
plt.imshow(files[0], cmap='gray', origin='lower', vmin=np.median(files[0])-2*np.std(files[0]),
                    vmax=np.median(files[0])+3*np.std(files[0]))
plt.subplot(132)
plt.imshow(files[1], cmap='gray', origin='lower', vmin=np.median(files[0])-2*np.std(files[0]),
                    vmax=np.median(files[0])+3*np.std(files[0]))
plt.subplot(133)
plt.imshow(files[2], cmap='gray', origin='lower', vmin=np.median(files[0])-2*np.std(files[0]),
                    vmax=np.median(files[0])+3*np.std(files[0]))
plt.show()

In [ ]:
dark = fits.getdata('./Output/Dark.fits')
bias = fits.getdata('./Output/Bias.fits')
flat= fits.getdata('./Output/Flats/%s_flat.fits' % filt)

In [ ]:
exp_time = fits.getheader(filenames[0])['EXPTIME']

reduced = (files-(dark-bias)*(exp_time/200)-bias)/((flat-bias)/np.median(flat-bias))
reduced = reduced/exp_time

In [ ]:
plt.figure(figsize=((15,15)))
plt.subplot(131)
plt.imshow(reduced[0], cmap='gray', origin='lower', vmin=np.median(reduced[0])-2*np.std(reduced[0]),
                    vmax=np.median(reduced[0])+3*np.std(reduced[0]))
plt.subplot(132)
plt.imshow(reduced[1], cmap='gray', origin='lower', vmin=np.median(reduced[0])-2*np.std(reduced[0]),
                    vmax=np.median(reduced[0])+3*np.std(reduced[0]))
plt.subplot(133)
plt.imshow(reduced[2], cmap='gray', origin='lower', vmin=np.median(reduced[0])-2*np.std(reduced[0]),
                    vmax=np.median(reduced[0])+3*np.std(reduced[0]))
plt.show()

# Find Feige 66

In [ ]:
filt_frames = np.empty_like(reduced)

for i in range(len(reduced)):
    filt_frames[i] = signal.medfilt(reduced[i],7)

In [ ]:
np.std(filt_frames[0])

In [ ]:
plt.figure(figsize=((15,15)))
plt.subplot(131)
plt.imshow(filt_frames[0], cmap='gray', origin='lower', vmin=np.median(filt_frames[0])-2*np.std(filt_frames[0]),
                    vmax=np.median(filt_frames[0])+3*np.std(filt_frames[0]))
plt.subplot(132)
plt.imshow(filt_frames[1], cmap='gray', origin='lower', vmin=np.median(filt_frames[1])-2*np.std(filt_frames[1]),
                    vmax=np.median(filt_frames[1])+3*np.std(filt_frames[1]))
plt.subplot(133)
plt.imshow(filt_frames[2], cmap='gray', origin='lower', vmin=np.median(filt_frames[2])-2*np.std(filt_frames[2]),
                    vmax=np.median(filt_frames[2])+3*np.std(filt_frames[2]))
plt.show()

In [ ]:
mask = np.zeros_like(filt_frames)
median=np.ndarray(len(filt_frames))
std=np.ndarray(len(filt_frames))

for i in range(len(filt_frames)):
    median[i] = np.median(filt_frames[i])
    std[i] = np.std(filt_frames[i])

for k in range(len(filt_frames)):
    for i in range(1024):
        for j in range(1024):
            if filt_frames[k][i][j] > median[k]+5*std[k]:
                mask[k][i][j] = 1

In [ ]:
plt.figure(figsize=((15,15)))
plt.subplot(131)
plt.imshow(mask[0], origin='lower', cmap='gray')
plt.subplot(132)
plt.imshow(mask[1], origin='lower', cmap='gray')
plt.subplot(133)
plt.imshow(mask[2], origin='lower', cmap='gray')
plt.show()

In [ ]:
star = [0,0,0]
max_pos = np.ndarray((len(filt_frames),2))

for i in range(len(filt_frames)):
    labels,num = snd.label(mask[i] == 1, np.ones((3,3)))
    centers = snd.center_of_mass(mask[i],labels,range(1,num+1))
    max_pos[i] = snd.maximum_position(filt_frames[i],labels)
    star[i] = centers

In [ ]:
r = [5,8,11,14,17,20,30]

flux = np.zeros((len(filt_frames),len(r)))
reduced_copy = np.copy(reduced)

for l in range(len(filt_frames)):
    for k in range(len(r)):
        for i in range(int(max_pos[l][0])-40,int(max_pos[l][0]+40)):
            for j in range(int(max_pos[l][1])-40,int(max_pos[l][1]+40)):
                if ((i-max_pos[l][0])**2+(j-max_pos[l][1])**2) < (r[k]**2):
                    flux[l][k] = flux[l][k] + reduced[l][i][j]


In [ ]:
plt.figure(figsize=((15,15)))

#Frame 1
plt.subplot(131)

circle1=plt.Circle((max_pos[0][1],max_pos[0][0]),r[0],color='r',fill=False)
plt.gcf().gca().add_artist(circle1)
circle2=plt.Circle((max_pos[0][1],max_pos[0][0]),r[1],color='r',fill=False)
plt.gcf().gca().add_artist(circle2)
circle3=plt.Circle((max_pos[0][1],max_pos[0][0]),r[2],color='r',fill=False)
plt.gcf().gca().add_artist(circle3)
circle4=plt.Circle((max_pos[0][1],max_pos[0][0]),r[3],color='r',fill=False)
plt.gcf().gca().add_artist(circle4)
circle5=plt.Circle((max_pos[0][1],max_pos[0][0]),r[4],color='r',fill=False)
plt.gcf().gca().add_artist(circle5)
circle6=plt.Circle((max_pos[0][1],max_pos[0][0]),r[5],color='r',fill=False)
plt.gcf().gca().add_artist(circle6)
circle7=plt.Circle((max_pos[0][1],max_pos[0][0]),r[6],color='r',fill=False)
plt.gcf().gca().add_artist(circle7)

plt.imshow(reduced[0], cmap='gray', origin='lower', vmin=np.median(reduced[0])-2*np.std(reduced[0]),
                    vmax=np.median(reduced[0])+3*np.std(reduced[0]))

plt.xlim(max_pos[0][1]-50,max_pos[0][1]+50)
plt.ylim(max_pos[0][0]-50,max_pos[0][0]+50)


#Frame 2
plt.subplot(132)

circle1=plt.Circle((max_pos[1][1],max_pos[1][0]),r[0],color='r',fill=False)
plt.gcf().gca().add_artist(circle1)
circle2=plt.Circle((max_pos[1][1],max_pos[1][0]),r[1],color='r',fill=False)
plt.gcf().gca().add_artist(circle2)
circle3=plt.Circle((max_pos[1][1],max_pos[1][0]),r[2],color='r',fill=False)
plt.gcf().gca().add_artist(circle3)
circle4=plt.Circle((max_pos[1][1],max_pos[1][0]),r[3],color='r',fill=False)
plt.gcf().gca().add_artist(circle4)
circle5=plt.Circle((max_pos[1][1],max_pos[1][0]),r[4],color='r',fill=False)
plt.gcf().gca().add_artist(circle5)
circle6=plt.Circle((max_pos[1][1],max_pos[1][0]),r[5],color='r',fill=False)
plt.gcf().gca().add_artist(circle6)
circle7=plt.Circle((max_pos[1][1],max_pos[1][0]),r[6],color='r',fill=False)
plt.gcf().gca().add_artist(circle7)

plt.imshow(reduced[1], cmap='gray', origin='lower', vmin=np.median(reduced[1])-2*np.std(reduced[1]),
                    vmax=np.median(reduced[1])+3*np.std(reduced[1]))

plt.xlim(max_pos[1][1]-50,max_pos[1][1]+50)
plt.ylim(max_pos[1][0]-50,max_pos[1][0]+50)

#Frame 3
plt.subplot(133)

circle1=plt.Circle((max_pos[2][1],max_pos[2][0]),r[0],color='r',fill=False)
plt.gcf().gca().add_artist(circle1)
circle2=plt.Circle((max_pos[2][1],max_pos[2][0]),r[1],color='r',fill=False)
plt.gcf().gca().add_artist(circle2)
circle3=plt.Circle((max_pos[2][1],max_pos[2][0]),r[2],color='r',fill=False)
plt.gcf().gca().add_artist(circle3)
circle4=plt.Circle((max_pos[2][1],max_pos[2][0]),r[3],color='r',fill=False)
plt.gcf().gca().add_artist(circle4)
circle5=plt.Circle((max_pos[2][1],max_pos[2][0]),r[4],color='r',fill=False)
plt.gcf().gca().add_artist(circle5)
circle6=plt.Circle((max_pos[2][1],max_pos[2][0]),r[5],color='r',fill=False)
plt.gcf().gca().add_artist(circle6)
circle7=plt.Circle((max_pos[2][1],max_pos[2][0]),r[6],color='r',fill=False)
plt.gcf().gca().add_artist(circle7)

plt.imshow(reduced[2], cmap='gray', origin='lower', vmin=np.median(reduced[2])-2*np.std(reduced[2]),
                    vmax=np.median(reduced[2])+3*np.std(reduced[2]))

plt.xlim(max_pos[2][1]-50,max_pos[2][1]+50)
plt.ylim(max_pos[2][0]-50,max_pos[2][0]+50)

plt.show()

In [ ]:
print flux

In [ ]:
F66 = np.ndarray((len(flux),len(r)))

for i in range(len(flux)):
    for j in range(len(r)):
        F66[i][j] = flux[i][j]-((flux[i][len(r)-1]-flux[i][len(r)-2])/(np.pi*(r[len(r)-1]**2-r[len(r)-2]**2))*(np.pi*(r[j]**2)))

print F66

In [ ]:
fig = plt.figure(figsize=(10,5))
for i in range(len(flux)):
    plt.plot(r,F66[i], color='k', label='Frame %d' % (i+1))
    
plt.ylabel('Feige 66 Flux (ADU/s)')
plt.xlabel('Radius (pix)')
plt.title('Flux from Feige 66 vs. Radius', fontsize=20)
fig.savefig('./Output/Figures/F66_growth.pdf', bbox_inches='tight')
plt.show()

In [ ]:
ext_coef = {'B':0.302,'R':0.104,'I':0.051,'V':0.164,'Ha':0.104}
F66_mag = {'B':10.26,'R':10.642,'I':10.809,'V':10.59,'Ha':11.024}

In [ ]:
airmass = np.mean([fits.getheader(image)['AIRMASS'] for image in filenames])
airmass

In [ ]:
F66_mag[filt]+2.5*np.log10(np.mean(F66[:,3]))+ext_coef[filt]*airmass


In [ ]:
zpt = {'B':20.495713252099467,'R':20.614211648453246,'I':19.197490584726474,'V':20.647121539787459,'Ha':17.575096769996414}

# Error

In [ ]:
sigma_shot=np.sqrt(F66[:,3])
sigma_shot

In [ ]:
sigma_sky = [0,0,0]

for i in range(len(flux)):   
    sigma_sky[i] = (flux[i][6]-flux[i][5])/(np.pi*(r[6]**2-r[5]**2))*(np.pi*r[3]**2)

print sigma_sky

In [ ]:
sigma_flux = [0,0,0]
for i in range(len(sigma_flux)):
    sigma_flux[i] = np.sqrt(sigma_shot[i]**2+sigma_sky[i]**2)
sigma_flux

In [ ]:
sigma_m = [0,0,0]
for i in range(len(sigma_m)):
    sigma_m[i] = (2.5/np.log(10))*sigma_flux[i]/F66[i,3]
sigma_m

In [ ]:
sigma_m = np.sqrt(sigma_m[0]**2+sigma_m[1]**2+sigma_m[2]**2)
sigma_m

In [ ]:
z_err = {'B':0.46886509632532331,'R':1.418729232296043,'I':1.3018832822952522,'V':1.2930335974139877,'Ha':0.89876087285199369}